# Structured Output

In [ ]:
from devtools import debug
from dotenv import load_dotenv

load_dotenv(verbose=True)

!export PYTHONPATH=":./python"

### Method 1 : provide instruction in the prompt

In [ ]:
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field

In [ ]:
"""
The usual "tell me a joke" LLM call.
"""

from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt


class Joke(BaseModel):
    the_joke: str = Field(description="a good joke")
    explanation: str = Field(description="explain why it's funny")
    rate: float = Field(description="rate how the joke is funny between 0 and 5")


parser = PydanticOutputParser(pydantic_object=Joke)

prompt_with_format = """
    tell me  a joke on {topic}     
    --- 
    {format_instructions}"""

structured_prompt = def_prompt(user=prompt_with_format).partial(
    format_instructions=parser.get_format_instructions(),
)

LLM_ID = None
structured_joke = structured_prompt | get_llm(llm_id=LLM_ID, json_mode=True) | parser

r = structured_joke.invoke({"topic": "cat"})
debug(r)

In [ ]:
debug(structured_prompt)

In [ ]:
# You can have a look at the generated prompt:
print(structured_prompt.invoke({"topic": "cat"}).messages[0].content)

### Method #2 : Use "with_structured_output"  (bases on function calls)

In [ ]:
prompt = "tell me  a joke on {topic}"

# MODEL = None
MODEL = "gpt_4_azure"
chain = def_prompt(prompt) | get_llm(llm_id=MODEL).with_structured_output(Joke)
debug(chain.invoke(({"topic": "cat"})))

##  Assignement (Optional)
Rate the above joke.
Use https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/enum/ 


In [ ]:
from enum import Enum


class JokeRater(Enum):
    NOT_SO_GOOD = 0
    GOOD = 1
    VERY_GOOD = 2